# Support Vector Machine

**Disclaimer:** Some of the functions used in these notes were taken from [this online book](https://jakevdp.github.io/PythonDataScienceHandbook/).

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set()

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=50, centers=2,
                  random_state=0, cluster_std=0.60)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn');

### What is a *Hyperplane*?

In a $k$-dimensional space, a [hyperplane](https://en.wikipedia.org/wiki/Hyperplane) is a flat affine subspace of dimension $k-1$. For instance, in two dimensions, a hyperplane is a flat one-dimensional subspace - in other words, a line. In three dimensions, a hyperplane is a flat two-dimensional subspace - that is, a plane. In $k>3$ dimensions, it can be hard to visualize a hyperplane, but the notion of a $(k-1)$ -dimensional flat subspace still applies.

The mathematical definition of a hyperplane is quite simple. In two dimensions, a hyperplane is defined by the equation

$$
\beta_{0}+\beta_{1} X_{1}+\beta_{2} X_{2}=0
$$

for parameters $\beta_{0}, \beta_{1}$, and $\beta_{2}$. When we say that the above equation "defines" the hyperplane, we mean that any $X=\left(X_{1}, X_{2}\right)^{\prime}$ for which the equation holds is a point on the hyperplane. Note this equation is simply the equation of a line, since indeed in two dimensions a hyperplane is a line.

Clearly in the multidimensional case one has

$$
\begin{aligned}
\beta_{0}+\beta_{1} X_{1}+\beta_{2} X_{2}+\ldots+\beta_{k} X_{k}&=0\\
\mathbf{x}^{\prime}\boldsymbol{\beta}&=0,
\end{aligned}
$$

and this defines a $k$-dimensional hyperplane in the sense that if $X=[X_1,X_2,\ldots,X_k]^{\prime}\in\mathbb{R}^{k}$ satisfies $\mathbf{x}^{\prime}\boldsymbol{\beta}=0$, then it is said that $X$ *lies* on the hyperplane.

On the other hand, if
$$
\begin{aligned}
\beta_{0}+\beta_{1} X_{1}+\beta_{2} X_{2}+\ldots+\beta_{k} X_{k}&<0\\
\mathbf{x}^{\prime}\boldsymbol{\beta}&<0.
\end{aligned}
$$
then $X$ lies on the other side of the hyperplane. So we can think of the hyperplane as dividing $k$-dimensional space into two halves. One can easily determine on which side of the hyperplane a point lies by simply calculating the sign of the left hand side of $\mathbf{x}^{\prime}\boldsymbol{\beta}$.

<div>
<img src="img/91.png" width="700" align="center"/>
</div>
👆🏼 The hyperplane $1+2 X_{1}+3 X_{2}=0$ is shown. The blue region is
the set of points for which $1+2 X_{1}+3 X_{2}>0$, and the purple region is the set of
points for which $1+2 X_{1}+3 X_{2}<0$.

#### Classification using a *Separating* Hyperplane

<div>
<img src="img/92.png" width="1000" align="center"/>
</div>



Suppose we have observations $\{(y_{1},x_{1,1},x_{1,2}), (y_{2},x_{2,1},x_{2,2}),\ldots,(y_{n},x_{n1},x_{n2})\}$ and we know that these $n$ observations fall into two classes - that is $\{y_1,y_2,\ldots,y_n\}\in\{-1,1\}$ where -1 represents one class (<font color='purple'>purple</font>) and 1 the other class (<font color='blue'>blue</font>). *Left*: Three (out of many) possible separating hyperplanes. *Right*: Decision rule made by a *classifier* based on this particular hyperplane (black line). If a test observation falls in the blue portion of the grid, it will be assigned to the <font color='blue'>blue</font> case, and to the <font color='purple'>purple</font> otherwise.

In [ ]:
xfit = np.linspace(-1, 3.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plt.plot([0.6], [2.1], 'x', color='red', markeredgewidth=2, markersize=10)

for m, b in [(1, 0.65), (0.5, 1.6), (-0.2, 2.9)]:
    plt.plot(xfit, m * xfit + b, '-k')

plt.xlim(-1, 3.5);

Then a separating hyperplane for any $k$ has the property that

$$
\beta_{0}+\beta_{1} x_{i, 1}+\beta_{2} x_{i, 2}+\ldots+\beta_{k} x_{i, k}>0 \text { if } y_{i}=1
$$
and
$$
\beta_{0}+\beta_{1} x_{i, 1}+\beta_{2} x_{i, 2}+\ldots+\beta_{k} x_{i, k}<0 \text { if } y_{i}=-1\text{.}
$$
Equivalently, a separating hyperplane has the property that
$$
y_{i}\left(\beta_{0}+\beta_{1} x_{i, 1}+\beta_{2} x_{i, 2}+\ldots+\beta_{k} x_{i, k}\right)>0
$$
for all $i=1, \ldots, n$ since $y_i\in\{-1,1\}$.

<ins>Example</ins>: Imagine we are given a *test* observation $\mathbf{x}^\ast=\left[x_{1}^\ast,\ldots,x_{k}^\ast\right]^\prime$, then we 'assign' it to a class based on the sign of $\mathbf{x}^{\ast\prime}\boldsymbol{\beta}=:f(\mathbf{x}^\ast)$, i.e., if $f(\mathbf{x}^\ast)>0$ then we assign this test observation to class 1, and if $f(\mathbf{x}^\ast)<0$ then we assign it to class -1.

The **magnitude** of $f(\mathbf{x}^\ast)$ is also useful, since $f(\mathbf{x}^\ast)$ being far from zero makes us confident about its classification, but when $f(\mathbf{x}^\ast)$ is close to zero, then $\mathbf{x}^\ast$ is located near the hyperplane, and so we are less confident about the class assignment for it.

#### The Maximal Margin Classifier

<div class="foo">
    
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;| &nbsp; 
------ | -----
<div><img src="img/93.png" width="5500" align="center"/></div> | We can compute the (perpendicular) distance from each training observation to a given separating hyperplane; the smallest such distance is the minimal distance from the observations to the hyperplane, and is known as the *margin*. The maximal margin hyperplane is the separating hyperplane for which the margin is largest—that is, it is the hyperplane that has the farthest minimum distance to the training observations. We can then classify a test observation based on which side of the maximal margin hyperplane it lies. This is known as the **maximal margin classifier**. <ins>Example</ins>: we see that three training observations are equidistant from the maximal margin hyperplane and lie along the dashed lines indicating the width of the margin. These three observations are known as **support vectors**, since they are vectors in $k$-dimensional space ($k = 2$), and they "support" the maximal margin hyperplane in the sense vector that if these points were moved slightly then the maximal margin hyperplane would move as well.

</div>

In [ ]:
xfit = np.linspace(-1, 3.5)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')

for m, b, d in [(1, 0.65, 0.33), (0.5, 1.6, 0.55), (-0.2, 2.9, 0.2)]:
    yfit = m * xfit + b
    plt.plot(xfit, yfit, '-k')
    plt.fill_between(xfit, yfit - d, yfit + d, edgecolor='none',
                     color='#AAAAAA', alpha=0.4)

plt.xlim(-1, 3.5);

In [ ]:
from sklearn.svm import SVC # "Support vector classifier"
model = SVC(kernel='linear', C=1E10)
model.fit(X, y)

💻 The following fucntion will help us visualize the SVM decision boundaries:

In [ ]:
# This function will plot SVM decision boundaries for us
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=100, linewidth=1, facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(model);

💻 Printing the _support vectors_.

In [ ]:
model.support_vectors_

In [ ]:
def plot_svm(N=10, ax=None):
    X, y = make_blobs(n_samples=200, centers=2,
                      random_state=0, cluster_std=0.60)
    X = X[:N]
    y = y[:N]
    model = SVC(kernel='linear', C=1E10)
    model.fit(X, y)
    
    ax = ax or plt.gca()
    ax.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    ax.set_xlim(-1, 4)
    ax.set_ylim(-1, 6)
    plot_svc_decision_function(model, ax)

In [ ]:
from ipywidgets import interact, fixed
interact(plot_svm, N=[60, 120], ax=fixed(None));

⚠️ <ins>The separating hyperplane might not exist</ins>, and so there is no maximal margin classifier.

In [ ]:
from sklearn.datasets import make_circles
X, y = make_circles(100, factor=.1, noise=.1)

clf = SVC(kernel='linear').fit(X, y)

plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(clf, plot_support=False);

### Support Vector Classifier

The support vector classifier classifies a test observation depending on which side of a hyperplane it lies. The hyperplane is chosen to correctly separate most of the training observations into the two classes, but <ins>may misclassify a few observations</ins>. It is the solution to the optimization problem

$$
\begin{aligned}
& \underset{\beta_{0}, \beta_{1}, \ldots, \beta_{k}, \epsilon_{1}, \ldots, \epsilon_{n}}{\operatorname{maximimize}} M\\
& {\text { subject to } \sum_{j=1}^{k} \beta_{j}^{2}=1} \\
& {y_{i}\left(\beta_{0}+\beta_{1} x_{i,1}+\beta_{2} x_{i,2}+\ldots+\beta_{k} x_{i,k}\right) \geq M\left(1-\epsilon_{i}\right)} \\
& {\quad \epsilon_{i} \geq 0, \quad \sum_{i=1}^{n} \epsilon_{i} \leq C}
\end{aligned}
$$

where $C$ is a nonnegative tuning parameter, $M$ is the width of the margin (which we want to make as large as possible). The $\epsilon_{1}, \ldots, \epsilon_{n}$ are *slack variables* that allow observations to be on the wrong side of the margin.

1. The slack variable $\epsilon_{i}$ tells us where the $i$th observation is located relative to the hyperplane and the margin, i.e.,
 1. $\epsilon_{i}=0$ - the $i$th observation is on the correct side of the margin.
 2. $\epsilon_{i}>0$ - the $i$th observation is on the wrong side of the margin.
 3. $\epsilon_{i}>1$ - the $i$th observation is on the wrong side of the hyperplane.
2. The tuning parameter $C$  bounds the sum of the $\epsilon_{i}$'s, and can be considered a *tolerance* parameter.
 1. If $C=0$ there we are not allowing for violations so it must be the case that $\epsilon_{1}=\ldots=\epsilon_{n}=0$ in which case we have the maximal margin classifier (if it exists).
 2. For $C>0$ no more than $C$ observations can be on the wrong side of the hyperplane because in this case $\epsilon_{i}>1$ and we have that $\sum_{i=1}^{n} \epsilon_{i} \leq C$. As $C$ increases, we become more tolerant of violations to the margin, and so the margin will widen. Conversely, as $C$ decreases, we become less tolerant of violations to the margin and so the margin narrows.

📝 Please note that the `C` parameter in the `SVC` function acts contrary to what is described above, i.e., For very large `C`, the margin is hard, and points cannot lie in it. For smaller `C`, the margin is softer, and some points can lie within the margin.

In [ ]:
X, y = make_blobs(n_samples=100, centers=2,
                  random_state=0, cluster_std=0.8)

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for axi, C in zip(ax, [10.0, 0.1]):
    model = SVC(kernel='linear', C=C).fit(X, y)
    axi.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    plot_svc_decision_function(model, axi)
    axi.scatter(model.support_vectors_[:, 0],
                model.support_vectors_[:, 1],
                s=300, lw=1, facecolors='none');
    axi.set_title('C = {0:.1f}'.format(C), size=14)

#### Real Data Example

💻 This is a simulated data set of 10,000 observations and the objective is to build a machine that can predict loan ```default``` ('No' or 'Yes') based on the ```balance``` and ```income``` of the customers.

In [ ]:
import pandas as pd

df = pd.read_csv('https://r-data.pmagunia.com/system/files/datasets/dataset-63314.csv')

In [ ]:
print(df.head())
print(df.dtypes)

💻 We now proceed to match the 'No' and 'Yes' strings into numerical values for numerical calculations.

In [ ]:
df['default']  = df['default'].map({'Yes': 1, 'No': 0})
df['student'] = df['student'].map({'Yes': 1, 'No': 0})
print(df.head())
print(df.dtypes)

In [ ]:
import patsy
y, X = patsy.dmatrices('default ~ -1 + balance + income', data=df, return_type='dataframe')

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [ ]:
from sklearn import preprocessing
X_train_scaled = preprocessing.scale(X_train)
X_test_scaled = preprocessing.scale(X_test)

In [ ]:
from sklearn.svm import SVC

# Build your classifier
clf = SVC(kernel='linear', C=1)

# Train it on the entire training data set
clf.fit(X_train_scaled, y_train.values.ravel())

# Get predictions on the test set
y_pred = clf.predict(X_test_scaled)

# Assessing the fit
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

💻 We now choose the hyperparameter `C` by 3-fold CV over a grid of potential values for it.

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV, cross_val_score
svc = SVC(kernel='linear')

Cs = np.logspace(-6, -1, 10)
clf = GridSearchCV(estimator=svc, param_grid=dict(C=Cs),n_jobs=-1)
clf.fit(X_train_scaled, y_train.values.ravel())

In [ ]:
clf.best_score_

In [ ]:
clf.best_estimator_.C  

In [ ]:
# Prediction performance on test set is better that on train set
clf.score(X_test_scaled, y_test)

### Support Vector Machine

The support vector classifier is a natural approach for classification in the two-class setting, if the boundary between the two classes is linear. However, in practice we are sometimes faced with non-linear class boundaries, i.e.,

<div class="foo">
    
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;| &nbsp; 
------ | -----
<div><img src="img/98.png" width="5500" align="center"/></div> | Left: The observations fall into two classes, with a non-linear boundary between them. Right: The support vector classifier seeks a linear boundary, and consequently performs very poorly.

</div>

In [ ]:
X, y = make_circles(100, factor=.1, noise=.1,random_state=0)

clf = SVC(kernel='linear').fit(X, y)

plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(clf, plot_support=False);

It turns out that the solution to the support vector classifier problem involves only the [*inner products*](https://en.wikipedia.org/wiki/Inner_product_space) between the point $\mathbf{x}$ and the support vectors. So if $\mathcal{S}$ is the collection of indices of these support points, we can rewrite any solution function as

$$
f(\mathbf{x})=\beta_{0}+\sum_{i \in \mathcal{S}} \alpha_{i}\left\langle \mathbf{x}, \mathbf{x}_{i}\right\rangle,
$$

where $\langle \mathbf{a}, \mathbf{b}\rangle=\sum_{j=1}^{r} a_{j} b_{j}$. Therefore we can **generalize** this solution to 

$$
f(\mathbf{x})=\beta_{0}+\sum_{i \in \mathcal{S}} \alpha_{i} K\left(\mathbf{x}, \mathbf{x}_{i}\right),
$$

where $K$ is called a *kernel*.

<div class="foo">
    
&nbsp; &nbsp; &nbsp; | &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp;| &nbsp; &nbsp; &nbsp; 
 ------ | ------ | ------ 
  Name | Kernel | Parameter 
  Linear | $K\left(\mathbf{x}_{i}, \mathbf{x}_{i^{\prime}}\right)=\sum_{j=1}^{k} \mathbf{x}_{i j} \mathbf{x}_{i^{\prime} j}$ | ```C```
 Polynomial | $K\left(\mathbf{x}_{i}, \mathbf{x}_{i^{\prime}}\right)=\left(1+\sum_{j=1}^{k} x_{i j} x_{i^{\prime} j}\right)^{d}$ | $d$ (```degree```) and ```C```
 Radial | $K\left(\mathbf{x}_{i}, \mathbf{x}_{i^{\prime}}\right)=\exp \left(-\sigma \sum_{j=1}^{k}\left(x_{i j}-x_{i^{\prime} j}\right)^{2}\right)$ | $\sigma$ (```sigma```) and ```C```
 
</div>

In [ ]:
clf = SVC(kernel='rbf', C=1E6)
clf.fit(X, y)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
plot_svc_decision_function(clf)
plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
            s=300, lw=1, facecolors='none');

#### Real Data Example

In [ ]:
import patsy
y, X = patsy.dmatrices('default ~ -1 + balance + income', data=df, return_type='dataframe')

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [ ]:
import patsy
y, X = patsy.dmatrices('default ~ -1 + balance + income', data=df, return_type='dataframe')

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


In [ ]:
svcPoly = SVC(kernel='poly',degree=3)

Cs = np.logspace(-6, -1, 10)
clf = GridSearchCV(estimator=svcPoly, param_grid=dict(C=Cs),n_jobs=-1)
clf.fit(X_train_scaled, y_train.values.ravel())

print(clf.best_score_ )
print(clf.best_estimator_.C  )

# Prediction performance on test set is not better that on train set
clf.score(X_test_scaled, y_test)

In [ ]:
svcRadial = SVC(kernel='rbf',gamma=.01)

Cs = np.logspace(-6, -1, 10)
clf = GridSearchCV(estimator=svcRadial, param_grid=dict(C=Cs),n_jobs=-1)
clf.fit(X_train_scaled, y_train.values.ravel())

print(clf.best_score_ )
print(clf.best_estimator_.C  )

# Prediction performance on test set is not better that on train set
clf.score(X_test_scaled, y_test)